## Deforestation Risk Map API Query Demo
This demo allows the user to query our predicted map of deforestation. Right now this is our map of risk of deforestation between 2012-2017, but will soon be updated with our risk map of 2019-2024. Given an area of interest, the API will return summary statistics of the region including mean, min, max, and count of the probability of being deforested within the time period.

Our model is a prediction map of the risk of deforestation. The map was made by training a Random Forest based machine learning model on data from the University of Maryland Annual Tree Cover Loss dataset, using covariates covering accessibility, management, topography, and climate.  

This map and our API can be used by governments, land use planners, and nonprofit organizations to identify high risk areas of deforestation.

This API was put together by the World Resources Institute.

In [1]:
#Import needed libraries
#If you do not have the necessary library, uncomment the corresponding line to install via pip
#!pip install fiona
#!pip install requests
#!pip install zipfile
import os
import io
import sys
import requests
import fiona
import zipfile
%autosave 0


Autosave disabled


In [2]:
#To use this module, you need to have a shapefile to perform calculations over.
#If you do not have one prepared, you can run this module to download and extract a shapefile

r = requests.get('https://gfw-files.s3.amazonaws.com/ai4e/Tshopo.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(path='~/') # extract to folder
filenames =  z.namelist()

shapefile = [x for x in filenames if '.shp' in x][0]
path_to_shapefile = os.path.join('~/',shapefile)
print('Copy and paste this file in the form below: {}'.format(path_to_shapefile))

Copy and paste this file in the form below: ~/Tshopo.shp


In [3]:
#Use the form below to enter the path to your shapefile or geojson. This file should be on your local computer
#For example: /Users/kristine/WRI/MachineLearning/Deforestation/DRC_Provinces/Tshopo/Tshopo.shp
path_to_shapefile = input("Enter the path to your shapefile: ") 
print('The following input file was selected: {}\n'.format(path_to_shapefile))

#First check if the shapefile has any errors:    
try:
    src = fiona.open(path_to_shapefile)
    print('Inputted shapefile is okay!')
except:
    print('Inputted shapefile is not formatted correctly and has erroneous geometries. Please fix the geometries and try again.')
    

Enter the path to your shapefile: ~/Tshopo.shp
The following input file was selected: ~/Tshopo.shp

Inputted shapefile is okay!


In [4]:
#Define API querying packages
def get_my_api(URL):
    
    try:           
        r = requests.get(URL)
        if r.ok:
            return r.content.decode('utf-8')
        else:
            return None

    except Exception as e:
        print('Exception running API: {}'.format(e))
        
def post_my_api(URL,myobj):
    x = requests.post(URL, json = myobj)
    return x.content.decode('utf-8')


In [5]:
#Check that the API is running!
result = get_my_api('http://40.88.19.41/v1/my_api/tasker/echo/yes')
print('Is it running?')
print(result)

Is it running?
Echo: yes


In [6]:
#Process the shapefile to convert it to a json to send to API
print('The following file was selected: {}'.format(path_to_shapefile))
features = []
with fiona.open(path_to_shapefile) as src:
    for f in src:
        features.append(f)
content = {'features':features}

#Post query to API
task_id_string = post_my_api('http://40.88.19.41/v1/my_api/tasker/example',content)
task_id = int(task_id_string.split(':')[1])

#Get information on task
result = get_my_api('http://40.88.19.41/v1/my_api/tasker/task/{}'.format(task_id))
print(result)


The following file was selected: ~/Tshopo.shp
{"TaskId": 4439, "Status": "Writing to geojson", "Timestamp": "2019-11-01 14:34:29", "Endpoint": "uri"}



In [10]:
#Run this cell to get the status of your query! For large areas it takes a couple of minutes to get
#the zonal statistics. Run this cell multiple times until it is completed.
print('This may take a couple of minutes')
result = get_my_api('http://40.88.19.41/v1/my_api/tasker/task/{}'.format(task_id))
print(result)

This may take a couple of minutes
{"TaskId": 4439, "Status": "Completed: Here are the statistics for your region(s): [{'min': 0.0, 'max': 0.9523997902870178, 'mean': 0.07448942288381855, 'count': 260645005}]", "Timestamp": "2019-11-01 14:34:29", "Endpoint": "uri"}

